In [3]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import scipy.special as sp

# Decentrality

We can measure decentrality by calculating the expected Shannon entropy of the resulting distribution of coins among the peers

In [26]:
p = 4
N_x, N = np.ones(p), sum(N_x) 
print(N_x, N)
-sum(N_x/N*(sp.psi(N_x + 1) - sp.psi(N + 1))), np.log(p)

[1. 1. 1. 1.] 4.0


(1.083333333333333, 1.3862943611198906)

# Blockchain queue

Blockchain efficiency is characterized by the throughput (how many transactions are confirmed per time unit) and latency (the average confirmation time of a transaction). These quantities of interest are study within the frame of a queueing model. The pending transactions arrive at a Poisson rate to form a queue, a fixed number $b$ (the first which entered the queue) will be processed in a block. The block generation time has distribution $G$ on $\mathbb{R}_+$. The result is a $M/G^b/1$ queue. We are going to simulate trajectories of this process to study the average confirmation time and the number of pending transaction in the queue at stationarity. Let us assume that $G$ is exponential, which corresponds to the empirical data on the block arrival time in the bitcoin blockchain.

Denote by $(S_i)$ the transaction arrival time and $(T_i)$ the blockarrival times.We denote by $N_t$ the number of transaction and $M_t$ the number of block up to time $t>0$. Both of the processes are Poisson process with respective intensity $\lambda$ and $\mu$.

In [184]:
# Parameters of the transaction arrival, the block size and the service time
μ, λ, b = 3, 2, 3
# We set a time horizon, the higher the better to reach stationarity
t = 10000
N_t, M_t = np.random.poisson(λ * t), np.random.poisson(μ * t)
S, T = np.sort(np.random.uniform(low = 0, high = t, size = N_t)), np.sort(np.random.uniform(low = 0, high = t, size = M_t))
T = np.insert(T, 0, 0)

In [185]:
queue = np.array([])
server = []
conf_time = []
for k in range(len(T)-1):    
    queue = np.append(queue, S[np.logical_and(S>T[k], S < T[k+1])])
    if len(queue) > b:
        conf_time.append(np.sum(T[k+1] - queue[:b]))
        server.append(b)
        queue = queue[b:]
    else:
        if len(queue) > 0:
            conf_time.append(np.sum(T[k+1] - queue))
            server.append(len(queue))
            queue = np.array([])
        else:
            server.append(0)


In [186]:
print(np.sum(conf_time) / sum(S < T[-1]), np.mean(server))

0.35569455176670367 0.661092161472994


In [187]:
import scipy as sc
def f(x):
    return(- λ / μ *x**(b+1) + x**b * (λ / μ + 1) - 1 )

sol = sc.optimize.root_scalar(f, bracket=[1.00001, 100000], method='brentq')
p = 1/sol.root
geom_pmf = np.array([p**k * (1-p) for k in np.arange(0,b+1)])
latency, throughput = p/(1-p)/λ, sum(np.arange(0,b+1) * geom_pmf) + p**(b+1)*b
print(latency, throughput)

0.3596746103626663 0.6666666666666665
